<a href="https://colab.research.google.com/github/drools-git/TestGIT/blob/master/WhisperYouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're looking at this on GitHub and new to Python Notebooks or Colab, click the Google Colab badge above 👆


#**Creating YouTube transcripts with OpenAI's Whisper model**

📺 Getting started video: https://youtu.be/kENRf82_RQs

*Colab beginner notes:*
<br>
1. These files are being loaded on a virtual machine in the cloud. Nothing is being downloaded to your computer (except for the transcript when you click to download it.) When you close this session the instance will be erased.
<br>
2. The run button is visible when you move your mouse close to the left edge of the code block. It looks kind of like this: ▶️ ...but round...and white on black...so nothing like this. You'll know it when you see it.

###**Note: For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Step 1.** Follow the instructions in each block and select the options you want
<br>
**Step 2.** Get the url of the video you want to transcribe
<br>
**Step 3.** Refresh the folder on the left and download your transcript
<br>
**Step 4.** Go to your YouTube account and upload the transcript to the video it came from and use "autosync."

That's it!

Have a question? Hit me up on Twitter:[ @AndrewMayne](https://twitter.com/andrewmayne)

<br>



---


**What is this?**
<br>
This is a Python notebook that creates a transcript from a YouTube url using OpenAI's Whisper transcription model that you can then upload to YouTube using the autosync feature to create captions.
<br>  
**What is OpenAI's Whisper model?**
<br>
Whisper is an automatic speech recognition (ASR) neural net created by OpenAI that transcribes audio at close to human level.
<br>
<br>
**Why use this?**
<br>
The quality of the OpenAI Whisper model is amazing (I am slightly biased, but seriously, check it out.) You can also use it to transcribe in other languages.
<br>
<br>
**What do the different model sizes do?**
<br>
Each model size has an improvement in quality – especially with different languages. I've found that for a YouTube video with clear speech, the base model works really well. If you see transcription errors, you can try a larger model.
<br>
<br>
**Do I need timestamps?**
<br>
Nope. YouTube's autosync function will match the text to the spoken words and syncs up really well. All you need is each spoken sentence in a .txt file.
<br>
<br>
**How do I do this?**
<br>
Just follow each step. If you've never used Colab of a Python notebook, don't panic. It's super easy and runs in the cloud.
<br>
<br>
**Does this cost anything to use?**
<br>
Nope. You can use Colab for free and Whisper is an open source model.
<br>
<br>
[Tips for creating a YouTube transcript file](https://support.google.com/youtube/answer/2734799?hl=en)
<br>
[Information on OpenAI's Whisper model](https://openai.com/blog/whisper/)
<br>
[OpenAI's Whisper GitHub page](https://github.com/openai/whisper)
<br>












In [1]:
"""
1. Click the start button in the upper left side of this block to load the necessary libraries

You will need to run this every time you reload this notebook.
"""

!pip install youtube_dl
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import re
import youtube_dl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-64ju1fv9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-64ju1fv9
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803583 sha256=845d18381cf274a82afa0270b6350513e1bfec719a9d617bf30e4f27a957f545
  Stored in directory: /tmp/pip-ephem-wheel-cache-fowwmxh8/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
G

In [2]:
"""
2. Select the model you want to use.

Base works really well so it's the default.

(For multilingual, remove ".en" from the model name.)

Click the run button after you've made your choice (or left it at default.)
"""

# model = whisper.load_model("tiny.en")
model = whisper.load_model("base.en")
# model = whisper.load_model("small.en")
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 117MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [4]:
!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2024.12.6-py3-none-any.whl size=2922227 sha256=03b63d3ecc78902b3336ff49e1cbe808066d4cf516e88e63107fbdbcc0e0f3f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4dur0fk/wheels/4c/91/d1/c5369304e2f7afb660bb6eee093af5a7d3c0ea05a3c1e8c797
Successfully built yt-dlp


In [6]:
""" good
3. Click the run button and input your YouTube URL in the box below then click enter.

You can use this one to test: https://www.youtube.com/watch?v=CnT-Na1IeVI

The video will be loaded and the audio extracted (this is usually the longest part of the process.)

Your transcript will appear in the folder on the left (you may have to refresh the folder to see it.)

You can download the file when it's completed and upload it on your video's detail page using "autosync."
"""
import yt_dlp # Import yt-dlp instead of youtube-dl

# This will prompt you for a YouTube video URL
url = input("Enter a YouTube video URL: ")

# Create a youtube-dl options dictionary
ydl_opts = {
    # Specify the format as bestaudio/best
    'format': 'bestaudio/best',
    # Specify the post-processor as ffmpeg to extract audio and convert to mp3
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    # Specify the output filename as the video title
    'outtmpl': '%(title)s.%(ext)s',
}

# Download the video and extract the audio using yt_dlp
with yt_dlp.YoutubeDL(ydl_opts) as ydl: # Use yt_dlp.YoutubeDL
    ydl.download([url])

# Get the path of the file
file_path = ydl.prepare_filename(ydl.extract_info(url, download=False))
file_path = file_path.replace('.webm', '.mp3')
file_path = file_path.replace('.m4a', '.mp3')

# Get the duration
duration = librosa.get_duration(filename=file_path)
start = time.time()
result = model.transcribe(file_path)
end = time.time()
seconds = end - start

print("Video length:", duration, "seconds")
print("Transcription time:", seconds)

# Split result["text"]  on !,? and . , but save the punctuation
sentences = re.split("([!?.])", result["text"])

# Join the punctuation back to the sentences
sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
text = "\n\n".join(sentences)
for s in sentences:
  print(s)

# Save the file as .txt
name = "".join(file_path) + ".txt"
with open(name, "w") as f:
  f.write(text)

print("\n\n", "-"*100, "\n\nYour transcript is here:", name)

Enter a YouTube video URL: https://www.youtube.com/watch?v=xXCBz_8hM9w
[youtube] Extracting URL: https://www.youtube.com/watch?v=xXCBz_8hM9w
[youtube] xXCBz_8hM9w: Downloading webpage
[youtube] xXCBz_8hM9w: Downloading ios player API JSON
[youtube] xXCBz_8hM9w: Downloading mweb player API JSON
[youtube] xXCBz_8hM9w: Downloading player 62ccfae7
[youtube] xXCBz_8hM9w: Downloading m3u8 information
[info] xXCBz_8hM9w: Downloading 1 format(s): 251
[download] Destination: How To Build The Future： Sam Altman.webm
[download] 100% of   37.00MiB in 00:00:01 at 25.04MiB/s  
[ExtractAudio] Destination: How To Build The Future： Sam Altman.mp3
Deleting original file How To Build The Future： Sam Altman.webm (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=xXCBz_8hM9w
[youtube] xXCBz_8hM9w: Downloading webpage
[youtube] xXCBz_8hM9w: Downloading ios player API JSON
[youtube] xXCBz_8hM9w: Downloading mweb player API JSON
[youtube] xXCBz_8hM9w: Downloading m3u8 information


<ipython-input-6-2823eeaa100f>:41: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


Video length: 2811.5185625 seconds
Transcription time: 125.3371114730835
 We said from the very beginning we were going to go after AGI.
 At a time when in the field you weren't allowed to say that, because that just seemed impossibly crazy.
 I remember a rash of criticism for you guys at that moment.
 We really wanted to push on that.
 And we were far less resourced than the mind and others.
 And so we said, okay, they're going to try a lot of things and we've just got to pick one and really concentrate.
 And that's how we can win here.
 Most of the world still does not understand the value of like a fairly extreme level of conviction on one bet.
 That's why I'm so excited for startups right now.
 It is because the world is still sleeping on all this to such an astonishing degree.
 We have a real treat for you today.
 Sam Altman, thanks for joining us.
 Thanks, Gary.
 This is actually a reboot of your series, How to Build the Future.
 And so welcome back to the series that you started

In [5]:
""" bad
3. Click the run button and input your YouTube URL in the box below then click enter.

You can use this one to test: https://www.youtube.com/watch?v=CnT-Na1IeVI

The video will be loaded and the audio extracted (this is usually the longest part of the process.)

Your transcript will appear in the folder on the left (you may have to refresh the folder to see it.)

You can download the file when it's completed and upload it on your video's detail page using "autosync."
"""

# This will prompt you for a YouTube video URL
url = input("Enter a YouTube video URL: ")

# Create a youtube-dl options dictionary
ydl_opts = {
    # Specify the format as bestaudio/best
    'format': 'bestaudio/best',
    # Specify the post-processor as ffmpeg to extract audio and convert to mp3
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    # Specify the output filename as the video title
    'outtmpl': '%(title)s.%(ext)s',
}

# Download the video and extract the audio
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

# Get the path of the file
file_path = ydl.prepare_filename(ydl.extract_info(url, download=False))
file_path = file_path.replace('.webm', '.mp3')
file_path = file_path.replace('.m4a', '.mp3')

# Get the duration
duration = librosa.get_duration(filename=file_path)
start = time.time()
result = model.transcribe(file_path)
end = time.time()
seconds = end - start

print("Video length:", duration, "seconds")
print("Transcription time:", seconds)

# Split result["text"]  on !,? and . , but save the punctuation
sentences = re.split("([!?.])", result["text"])

# Join the punctuation back to the sentences
sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
text = "\n\n".join(sentences)
for s in sentences:
  print(s)

# Save the file as .txt
name = "".join(file_path) + ".txt"
with open(name, "w") as f:
  f.write(text)

print("\n\n", "-"*100, "\n\nYour transcript is here:", name)

Enter a YouTube video URL: https://www.youtube.com/watch?v=xXCBz_8hM9w
[youtube] xXCBz_8hM9w: Downloading webpage


ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


DownloadError: ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.